In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/animal_classification

/content/drive/MyDrive/animal_classification


In [4]:
from translate import translate
translate

{'cane': 'dog',
 'cavallo': 'horse',
 'elefante': 'elephant',
 'farfalla': 'butterfly',
 'gallina': 'chicken',
 'gatto': 'cat',
 'mucca': 'cow',
 'pecora': 'sheep',
 'scoiattolo': 'squirrel',
 'dog': 'cane',
 'elephant': 'elefante',
 'butterfly': 'farfalla',
 'chicken': 'gallina',
 'cat': 'gatto',
 'cow': 'mucca',
 'spider': 'ragno',
 'squirrel': 'scoiattolo'}

In [5]:
os.listdir("raw-img")

['cane',
 'cavallo',
 'elefante',
 'farfalla',
 'gallina',
 'gatto',
 'mucca',
 'pecora',
 'ragno',
 'scoiattolo']

In [12]:
for i in os.listdir("raw-img"):
  print(i,len(os.listdir("raw-img/"+i)))

dog 4863
horse 2623
elephant 1446
butterfly 2112
chicken 3098
cat 1668
cow 912
sheep 0
spider 0
squirrel 0


In [13]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("raw-img"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("raw-img/"+i)[:1000]:
    os.rename("raw-img/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("raw-img/"+i)[:400]:
    os.rename("raw-img/"+i+"/"+j,"test/"+i+"/"+j)

In [15]:
def img_Data(dir_path,target_size,batch,class_lst,preporcssing,):
  if preporcssing:
    gen_object = ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes=class_lst,
                                        shuffle=True))

In [17]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 6912 images belonging to 10 classes.
Found 2400 images belonging to 10 classes.


In [18]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
            input_shape=(224,224,3), alpha=1.0, include_top=False, weights='imagenet',
            input_tensor=None, pooling=None, classes=1000,
            classifier_activation='softmax')

9406464/9406464 [==============================] - 0s 0us/step


In [19]:
base_model.trainable = False

In [24]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [26]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 0.4582 - accuracy: 0.8537  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 3134s 227s/step - loss: 0.4582 - accuracy: 0.8537 - val_loss: 0.1455 - val_accuracy: 0.9546
Epoch 2/10
14/14 [==============================] - 64s 5s/step - loss: 0.1104 - accuracy: 0.9685 - val_loss: 0.1358 - val_accuracy: 0.9583
Epoch 3/10
14/14 [==============================] - 55s 4s/step - loss: 0.0701 - accuracy: 0.9777 - val_loss: 0.1296 - val_accuracy: 0.9608
Epoch 4/10
14/14 [==============================] - 53s 4s/step - loss: 0.0487 - accuracy: 0.9855 - val_loss: 0.1443 - val_accuracy: 0.9596
Epoch 5/10
14/14 [==============================] - 53s 4s/step - loss: 0.0335 - accuracy: 0.9918 - val_loss: 0.1324 - val_accuracy: 0.9629
Epoch 6/10
14/14 [==============================] - 54s 4s/step - loss: 0.0218 - accuracy: 0.9962 - val_loss: 0.1207 - val_accuracy: 0.9629
Epoch 7/10
14/14 [==============================] - 54s 4s/step - loss: 0.0152 - accuracy: 0.9986 - val_loss: 0.1194 - val_accuracy: 0.9642
Epoch 8/10
14/14 [=========

In [27]:
model.save('.mdl_wt.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
